pca_w2v_100features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/w2v_withstopwordchangelevels_100_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,1.467638,-9.276128,-4.237992,0.591256,-2.013045,0.886678,4.941392,-0.424598,2.596143,...,-4.914064e-10,-1.997267e-07,-7.875128e-08,2.416969e-07,-1.387301e-07,1.915291e-07,2.292694e-07,1.192044e-07,-5.068276e-08,1.234202e-07
1,1,8.304064,6.618901,-2.514401,12.659624,-0.155970,8.424951,-1.332061,-4.042820,1.950768,...,-6.434957e-08,-7.789896e-08,-8.616825e-08,4.747099e-08,-7.937243e-08,-6.998719e-08,-1.977387e-08,1.800776e-07,-4.194693e-08,-1.416106e-07
2,2,19.165034,25.044385,11.884638,10.388268,-1.873116,11.626532,3.339382,-1.328423,2.614332,...,-2.472812e-07,8.687178e-09,-1.884281e-08,1.592052e-07,7.982479e-09,6.601167e-08,7.458550e-08,2.456021e-07,1.498876e-07,1.403704e-07
3,3,-10.406558,-1.932083,-1.660323,6.791327,1.684361,-2.604143,1.832156,3.221879,-1.839563,...,2.702058e-07,-9.978360e-08,3.483484e-08,-4.923112e-08,-2.958196e-07,3.176674e-08,-4.979764e-08,1.093991e-07,-2.524984e-07,1.545161e-08
4,4,5.377139,1.434550,-6.143918,0.739852,3.788921,-8.998487,-6.004730,2.522262,2.181066,...,-2.277894e-07,2.323854e-07,-1.108795e-08,2.712446e-07,-2.592068e-07,1.470205e-07,8.766063e-08,2.274616e-07,2.246559e-07,8.342359e-08


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.467638,-9.276128,-4.237992,0.591256,-2.013045,0.886678,4.941392,-0.424598,2.596143,-1.849633,...,-4.914064e-10,-1.997267e-07,-7.875128e-08,2.416969e-07,-1.387301e-07,1.915291e-07,2.292694e-07,1.192044e-07,-5.068276e-08,1.234202e-07
1,8.304064,6.618901,-2.514401,12.659624,-0.155970,8.424951,-1.332061,-4.042820,1.950768,-0.589848,...,-6.434957e-08,-7.789896e-08,-8.616825e-08,4.747099e-08,-7.937243e-08,-6.998719e-08,-1.977387e-08,1.800776e-07,-4.194693e-08,-1.416106e-07
2,19.165034,25.044385,11.884638,10.388268,-1.873116,11.626532,3.339382,-1.328423,2.614332,-5.251433,...,-2.472812e-07,8.687178e-09,-1.884281e-08,1.592052e-07,7.982479e-09,6.601167e-08,7.458550e-08,2.456021e-07,1.498876e-07,1.403704e-07
3,-10.406558,-1.932083,-1.660323,6.791327,1.684361,-2.604143,1.832156,3.221879,-1.839563,-4.273462,...,2.702058e-07,-9.978360e-08,3.483484e-08,-4.923112e-08,-2.958196e-07,3.176674e-08,-4.979764e-08,1.093991e-07,-2.524984e-07,1.545161e-08
4,5.377139,1.434550,-6.143918,0.739852,3.788921,-8.998487,-6.004730,2.522262,2.181066,-0.133521,...,-2.277894e-07,2.323854e-07,-1.108795e-08,2.712446e-07,-2.592068e-07,1.470205e-07,8.766063e-08,2.274616e-07,2.246559e-07,8.342359e-08


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 100), (524, 100))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3932862041152744


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.37377,0.37377,0.636644,0.315938,0.291985,0.291985,0.387423,0.227276


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    0.9972    0.9986       361
           2     0.9982    1.0000    0.9991       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         0.9992      1220
   macro avg     0.9994    0.9991    0.9992      1220
weighted avg     0.9992    0.9992    0.9992      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3818    0.1338    0.1981       157
           2     0.4653    0.8776    0.6082       237
           3     0.2727    0.0462    0.0789       130

    accuracy                         0.4485       524
   macro avg     0.3733    0.3525    0.2951       524
weighted avg     0.3925    0.4485    0.3540       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.37377,0.37377,0.636644,0.315938,0.291985,0.291985,0.387423,0.227276
1,RandomForest,0.99918,0.99918,0.999182,0.999180,0.448473,0.448473,0.392523,0.354022


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:

classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5579    0.3740    0.4478       361
           2     0.5435    0.7685    0.6367       553
           3     0.5459    0.3497    0.4263       306

    accuracy                         0.5467      1220
   macro avg     0.5491    0.4974    0.5036      1220
weighted avg     0.5483    0.5467    0.5280      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2571    0.1720    0.2061       157
           2     0.4457    0.6414    0.5260       237
           3     0.3205    0.1923    0.2404       130

    accuracy                         0.3893       524
   macro avg     0.3411    0.3352    0.3241       524
weighted avg     0.3582    0.3893    0.3593       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.596543,0.451082,0.396947,0.396947,0.387459,0.349502
1,RandomForest,0.999180,0.999180,0.999182,0.999180,0.421756,0.421756,0.383025,0.343209
2,Adaboost,0.546721,0.546721,0.548343,0.528021,0.389313,0.389313,0.358169,0.359273


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9607    0.8116    0.8799       361
           2     0.7959    0.9946    0.8842       553
           3     0.9688    0.7092    0.8189       306

    accuracy                         0.8689      1220
   macro avg     0.9085    0.8385    0.8610      1220
weighted avg     0.8880    0.8689    0.8666      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2278    0.1146    0.1525       157
           2     0.4431    0.7553    0.5585       237
           3     0.3171    0.1000    0.1520       130

    accuracy                         0.4008       524
   macro avg     0.3293    0.3233    0.2877       524
weighted avg     0.3473    0.4008    0.3360       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.596543,0.451082,0.396947,0.396947,0.387459,0.349502
1,RandomForest,0.999180,0.999180,0.999182,0.999180,0.421756,0.421756,0.383025,0.343209
2,Adaboost,0.546721,0.546721,0.548343,0.528021,0.389313,0.389313,0.358169,0.359273
3,GBM,0.868852,0.868852,0.888027,0.866555,0.400763,0.400763,0.347327,0.336031


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3884    0.3906    0.3895       361
           2     0.5448    0.4069    0.4658       553
           3     0.3401    0.4935    0.4027       306

    accuracy                         0.4238      1220
   macro avg     0.4244    0.4303    0.4193      1220
weighted avg     0.4472    0.4238    0.4274      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2937    0.2675    0.2800       157
           2     0.4615    0.3291    0.3842       237
           3     0.2453    0.4000    0.3041       130

    accuracy                         0.3282       524
   macro avg     0.3335    0.3322    0.3228       524
weighted avg     0.3576    0.3282    0.3331       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.37377,0.37377,0.636644,0.315938,0.291985,0.291985,0.387423,0.227276
1,RandomForest,0.99918,0.99918,0.999182,0.999180,0.448473,0.448473,0.392523,0.354022
2,SVM,0.42377,0.42377,0.447182,0.427406,0.328244,0.328244,0.357602,0.333123


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4856    0.5152    0.5000       361
           2     0.5835    0.7143    0.6423       553
           3     0.5000    0.2614    0.3433       306

    accuracy                         0.5418      1220
   macro avg     0.5230    0.4970    0.4952      1220
weighted avg     0.5336    0.5418    0.5252      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2955    0.3312    0.3123       157
           2     0.4656    0.5148    0.4890       237
           3     0.2442    0.1615    0.1944       130

    accuracy                         0.3721       524
   macro avg     0.3351    0.3358    0.3319       524
weighted avg     0.3597    0.3721    0.3630       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.596543,0.451082,0.396947,0.396947,0.387459,0.349502
1,RandomForest,0.999180,0.999180,0.999182,0.999180,0.421756,0.421756,0.383025,0.343209
2,Adaboost,0.546721,0.546721,0.548343,0.528021,0.389313,0.389313,0.358169,0.359273
3,GBM,0.868852,0.868852,0.888027,0.866555,0.400763,0.400763,0.347327,0.336031
4,SVM,0.454098,0.454098,0.332715,0.287674,0.458015,0.458015,0.505363,0.294066
5,KNN,0.541803,0.541803,0.533580,0.525199,0.372137,0.372137,0.359712,0.362974


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4483    0.1801    0.2569       361
           2     0.4867    0.8282    0.6131       553
           3     0.3358    0.1471    0.2045       306

    accuracy                         0.4656      1220
   macro avg     0.4236    0.3851    0.3582      1220
weighted avg     0.4375    0.4656    0.4052      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2364    0.0828    0.1226       157
           2     0.4569    0.8059    0.5832       237
           3     0.2941    0.1154    0.1657       130

    accuracy                         0.4179       524
   macro avg     0.3291    0.3347    0.2905       524
weighted avg     0.3505    0.4179    0.3416       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.596543,0.451082,0.396947,0.396947,0.387459,0.349502
1,RandomForest,0.999180,0.999180,0.999182,0.999180,0.421756,0.421756,0.383025,0.343209
2,Adaboost,0.546721,0.546721,0.548343,0.528021,0.389313,0.389313,0.358169,0.359273
3,GBM,0.868852,0.868852,0.888027,0.866555,0.400763,0.400763,0.347327,0.336031
4,SVM,0.454098,0.454098,0.332715,0.287674,0.458015,0.458015,0.505363,0.294066
5,KNN,0.541803,0.541803,0.533580,0.525199,0.372137,0.372137,0.359712,0.362974
6,GNB,0.465574,0.465574,0.437494,0.405240,0.417939,0.417939,0.350455,0.341644
